In [227]:
# Function to extract the correct label mappings from the CSV file
def get_labels():
    eye_color_label = {'blue': 0, 'brown': 1, 'dark': 2, 'dark brown': 3, 'gray': 4, 'green': 5}
    hair_color_label = {'black': 0, 'blonde': 1, 'brown': 2, 'dark': 3, 'dark brown': 4, 'gray': 5, 'light brown': 6, 'red': 7}
    eyebrow_color_label = {'black': 0, 'blonde': 1, 'brown': 2, 'dark': 3, 'dark brown': 4, 'gray': 5}
    skin_tone_label = {'dark': 0, 'fair': 1, 'light': 2, 'medium': 3}
    return eye_color_label, hair_color_label, eyebrow_color_label, skin_tone_label

# Function to generate label mappings for each attribute
eye_color_mapping, hair_color_mapping, eyebrow_color_mapping, skin_tone_mapping = get_labels()


In [228]:
import cv2
import torch
import joblib
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load your classifiers
eye_color_classifier = joblib.load('eye_color_classifier.pkl')
hair_color_classifier = joblib.load('hair_color_classifier.pkl')
eyebrow_color_classifier = joblib.load('eyebrow_color_classifier.pkl')
skin_tone_classifier = joblib.load('skin_tone_classifier.pkl')

# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

# Initialize the camera feed
cap = cv2.VideoCapture(0)

# Get screen resolution
screen_height = 1080  # You can adjust this if needed
screen_width = 1920   # You can adjust this if needed

# Calculate new window size to maintain aspect ratio (half the screen height)
window_height = screen_height // 2
aspect_ratio = 16 / 9  # Adjust based on your webcam feed aspect ratio
window_width = int(window_height * aspect_ratio)

# Set the OpenCV window size (resizing the window)
cv2.namedWindow("Webcam Feed with Classifications", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Webcam Feed with Classifications", window_width, window_height)

while True:
    # Capture frame from webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Convert frame to PIL image (CLIP input format)
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image for CLIP
    inputs = clip_processor(images=pil_image, return_tensors="pt")

    # Get embeddings using CLIP
    with torch.no_grad():
        embeddings = clip_model.get_image_features(**inputs)
    
    # Flatten the embeddings (1D tensor)
    embeddings = embeddings.flatten().numpy()

    # Predict labels
    eye_color = eye_color_classifier.predict([embeddings])
    hair_color = hair_color_classifier.predict([embeddings])
    eyebrow_color = eyebrow_color_classifier.predict([embeddings])
    skin_tone = skin_tone_classifier.predict([embeddings])

    # Prepare labels to overlay vertically
    eye_color_inv = {v: k for k, v in eye_color_mapping.items()}
    hair_color_inv = {v: k for k, v in hair_color_mapping.items()}
    eyebrow_color_inv = {v: k for k, v in eyebrow_color_mapping.items()}
    skin_tone_inv = {v: k for k, v in skin_tone_mapping.items()}

    # Prepare labels to overlay vertically using the actual names
    labels = [
        f"Eye Color: {eye_color_inv.get(eye_color[0], 'Unknown')}",
        f"Hair Color: {hair_color_inv.get(hair_color[0], 'Unknown')}",
        f"Eyebrow Color: {eyebrow_color_inv.get(eyebrow_color[0], 'Unknown')}",
        f"Skin Tone: {skin_tone_inv.get(skin_tone[0], 'Unknown')}"
    ]

    # Starting position (top-right corner)
    text_x = 0  # 250 pixels from the right
    text_y = 30  # Initial y position
    line_height = 30  # Distance between lines

    # Overlay each label vertically
    for label in labels:
        cv2.putText(frame, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        text_y += line_height

    # Show the frame in a window
    cv2.imshow('Webcam Feed with Classifications', frame)

    # Press 'q' to quit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


c:\Users\alexi\anaconda3\envs\audio_env\lib\site-packages\torch\nn\modules\module.py:504: RuntimeWarning: coroutine 'Server.serve' was never awaited
  super().__setattr__("_forward_hooks_with_kwargs", OrderedDict())


In [230]:
import cv2
import requests

# URL of the FastAPI predict endpoint
url = "http://127.0.0.1:8000/predict"

# Read the image with OpenCV
image_path = r"D:\Mine\AI\BogoBeauty\img_align_celeba\000007.jpg"
img = cv2.imread(image_path)

# Encode the image as JPEG in memory
success, encoded_image = cv2.imencode(".jpg", img)
if not success:
    raise Exception("Could not encode image")

# Convert to bytes
image_bytes = encoded_image.tobytes()

# Prepare files dict for requests
files = {"file": ("image.jpg", image_bytes, "image/jpeg")}

# Send POST request
response = requests.post(url, files=files)

# Check response
if response.status_code == 200:
    print("Predictions:")
    print(response.json())
else:
    print(f"Error: {response.status_code}")
    print(response.text)


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002A16A8BAB20>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))